In [ ]:
%load_ext tensorboard

In [12]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras_tuner as kt
from keras.backend import backend

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

from tvDatafeed import TvDatafeed, Interval

import datetime

In [25]:
batch_size=32
dir_log="../models/"+ datetime.datetime.now().strftime("%d-%h-%H.%M")
dir_log

'../models/26-Nov-10.49'

In [4]:
username = 'rheyhantensor'
password = '$Acong091283'
tv = TvDatafeed(username, password)


In [46]:
btc_daily = tv.get_hist(symbol='BTCUSD',exchange='BITSTAMP',interval=Interval.in_daily, n_bars=100000)

In [ ]:
class createmodel:
    def __init__(self, train, test):
        self.epochs
        self.train
        self.test